In [ ]:
!pip install tensorflow==2.15

In [ ]:
!unzip copnn.zip

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import time
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler

from copnn.regression import run_regression
from copnn.simulation import Count
from copnn.modes.categorical import Categorical
from copnn.modes.longitudinal import Longitudinal
from copnn.modes.spatial import Spatial
from copnn.distributions import get_distribution

In [ ]:
# Rossmann Store Sales dataset from Kaggle: https://www.kaggle.com/competitions/rossmann-store-sales/
# Run rossmann_etl.R script
rossmann = pd.read_csv('rossmann_extreme.csv')
rossmann['Store'] = rossmann['Store'] - 1
cols_to_drop = ['date', 'year', 'Customers', 'Q9']
rossmann.drop(cols_to_drop, axis=1, inplace=True)
print(rossmann.shape)
rossmann.head()

In [ ]:
print(len(rossmann['Store'].unique()))
print(rossmann['Store'].max())

In [ ]:
rossmann.rename(columns={'Store': 'z0', 'ExtremeMonth': 'y'}, inplace=True)

In [ ]:
n_cats = len(rossmann['z0'].unique())

In [ ]:
def get_mode(mode_par):
    if mode_par == 'categorical':
        mode = Categorical()
    elif mode_par == 'longitudinal':
        mode = Longitudinal()
    elif mode_par == 'spatial':
        mode = Spatial()
    else:
        raise NotImplementedError(f'{mode_par} mode not implemented.')
    return mode

In [ ]:
y_type = 'binary'
mode = get_mode('longitudinal')
batch = 100
epochs = 100
patience = 5
n_sig2bs = 3
n_sig2bs_spatial = 0
est_cors = []
n_neurons = [10, 5]
activation = 'relu'
dropout = []
spatial_embedded_neurons = []
qs = [n_cats]
dist_matrix = None
q_spatial = None
Z_non_linear = False
Z_embed_dim_pct = 10
time2measure_dict = {t: i for i, t in enumerate(np.sort(rossmann['t'].unique()))}
pred_future = True # change this for future mode
spatial_embed_neurons = None
resolution = None
verbose = True
log_params = False
idx = None
shuffle = True
b_true = None
distributions = ['gaussian'] # gaussian is what COPNN expects for the binary probit model

In [ ]:
res = pd.DataFrame(columns=['pred_future', 'experiment', 'distribution', 'exp_type', 'auc_no_re', 'auc', 'sigma_e_est',
                            'sigma_b0_est', 'sigma_b1_est', 'sigma_b2_est', 'rho_est', 'nll_tr', 'nll_te', 'n_epoch', 'time'])
kf = KFold(n_splits=5, shuffle=True, random_state=40)
counter = Count().gen()

if pred_future:
  # test set is "the future" or those obs with largest t
  rossmann.sort_values(['t'], inplace=True)
  X, X_future, y, y_future = train_test_split(
      rossmann.drop('y', axis=1), rossmann['y'], test_size=0.2, shuffle=False)
  X.index = np.arange(X.shape[0])
  y.index = np.arange(X.shape[0])
else:
  X, y = rossmann.drop('y', axis=1), rossmann['y']

x_cols = [col for col in X.columns if col not in ['z0']]
x_cols_to_scale = [col for col in x_cols if col not in ['D1', 'D2', 't']]

In [ ]:
def iterate_reg_types(X_train, X_test, y_train, y_test, counter, i, verbose):
    print(f'  started ignore...')
    res_ignore = run_regression(
        X_train, X_test, y_train, y_test, qs, q_spatial, x_cols,
        batch, epochs, patience, n_neurons, dropout, activation, 'ignore',
        Z_non_linear, Z_embed_dim_pct, mode, y_type, n_sig2bs, n_sig2bs_spatial, est_cors,
        dist_matrix, time2measure_dict, spatial_embed_neurons, resolution, verbose,
        log_params, idx, shuffle, None, b_true)
    print('  finished ignore, auc: %.3f' % res_ignore.metric_mse)
    res.loc[next(counter)] = [pred_future, i, 'gaussian', 'ignore', res_ignore.metric_mse_no_re, res_ignore.metric_mse,
                              None, None, None, None, res_ignore.sig_ratio,
                              res_ignore.nll_tr, res_ignore.nll_te, res_ignore.n_epochs, res_ignore.time]
    print(f'  started ohe...')
    res_ohe = run_regression(
        X_train, X_test, y_train, y_test, qs, q_spatial, x_cols,
        batch, epochs, patience, n_neurons, dropout, activation, 'ohe',
        Z_non_linear, Z_embed_dim_pct, mode, y_type, n_sig2bs, n_sig2bs_spatial, est_cors,
        dist_matrix, time2measure_dict, spatial_embed_neurons, resolution, verbose,
        log_params, idx, shuffle, None, b_true)
    print('  finished ohe, auc: %.3f' % res_ohe.metric_mse)
    res.loc[next(counter)] = [pred_future, i, 'gaussian', 'ohe', res_ohe.metric_mse_no_re, res_ohe.metric_mse,
                              None, None, None, None, res_ohe.sig_ratio,
                              res_ohe.nll_tr, res_ohe.nll_te, res_ohe.n_epochs, res_ohe.time]
    print(f'  started embedding...')
    res_embed = run_regression(
        X_train, X_test, y_train, y_test, qs, q_spatial, x_cols,
        batch, epochs, patience, n_neurons, dropout, activation, 'embed',
        Z_non_linear, Z_embed_dim_pct, mode, y_type, n_sig2bs, n_sig2bs_spatial, est_cors,
        dist_matrix, time2measure_dict, spatial_embed_neurons, resolution, verbose,
        log_params, idx, shuffle, None, b_true)
    print('  finished embed, auc: %.3f' % res_embed.metric_mse)
    res.loc[next(counter)] = [pred_future, i, 'gaussian', 'embed', res_embed.metric_mse_no_re, res_embed.metric_mse,
                              None, None, None, None, res_embed.sig_ratio,
                              res_embed.nll_tr, res_embed.nll_te, res_embed.n_epochs, res_embed.time]
    print(f'  started lmmnn...')
    res_lmmnn = run_regression(
        X_train, X_test, y_train, y_test, qs, q_spatial, x_cols,
        batch, epochs, patience, n_neurons, dropout, activation, 'lmmnn',
        Z_non_linear, Z_embed_dim_pct, mode, y_type, n_sig2bs, n_sig2bs_spatial, est_cors,
        dist_matrix, time2measure_dict, spatial_embed_neurons, resolution, verbose,
        log_params, idx, shuffle, None, b_true)
    print('  finished lmmnn, auc: %.3f' % res_lmmnn.metric_mse)
    res.loc[next(counter)] = [pred_future, i, 'gaussian', 'lmmnn', res_lmmnn.metric_mse_no_re, res_lmmnn.metric_mse,
                              res_lmmnn.sigmas[0], res_lmmnn.sigmas[1][0], res_lmmnn.sigmas[1][1], res_lmmnn.sigmas[1][2], res_lmmnn.sig_ratio,
                              res_lmmnn.nll_tr, res_lmmnn.nll_te, res_lmmnn.n_epochs, res_lmmnn.time]
    for fit_dist in distributions:
        fit_dist = get_distribution(fit_dist)
        print(f'  started copnn with marginal: {fit_dist}...')
        res_copnn = run_regression(
            X_train, X_test, y_train, y_test, qs, q_spatial, x_cols,
            batch, epochs, patience, n_neurons, dropout, activation, 'copnn',
            Z_non_linear, Z_embed_dim_pct, mode, y_type, n_sig2bs, n_sig2bs_spatial, est_cors,
            dist_matrix, time2measure_dict, spatial_embed_neurons, resolution, verbose,
            log_params, idx, shuffle, fit_dist, b_true)
        print('  finished copnn, auc: %.3f' % res_copnn.metric_mse)
        res.loc[next(counter)] = [pred_future, i, fit_dist, 'copnn', res_copnn.metric_mse_no_re, res_copnn.metric_mse,
                                  res_copnn.sigmas[0], res_copnn.sigmas[1][0], res_copnn.sigmas[1][1], res_copnn.sigmas[1][2], res_copnn.sig_ratio,
                                  res_copnn.nll_tr, res_copnn.nll_te, res_copnn.n_epochs, res_copnn.time]
    res.to_csv('res.csv')

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    print('iteration %d' % i)
    if not pred_future:
      X_train, X_test, y_train, y_test = X.loc[train_index].copy(), X.loc[test_index].copy(), y[train_index], y[test_index]
    else:
      X_train, X_test, y_train, y_test = X.loc[train_index].copy(), X_future.copy(), y[train_index], y_future.copy()
    scaler = StandardScaler()
    X_train[x_cols_to_scale] = scaler.fit_transform(X_train[x_cols_to_scale])
    X_test[x_cols_to_scale] = scaler.transform(X_test[x_cols_to_scale])
    iterate_reg_types(X_train, X_test, y_train, y_test, counter, i, verbose)